In [3]:
import pandas as pd
import cv2
import networkx as nx
import numpy as np
import math
import random
from scipy.stats import binom
import os
import openpyxl
#import scienceplots
import matplotlib.pyplot as plt
SFHH = pd.read_csv('E:/Desktop/毕业论文/论文1/data/tij_SFHH/202210.csv',names = ['time','i','j'],header = None)
ij = pd.read_csv('E:/Desktop/毕业论文/论文1/data/tij_SFHH/202210label.csv',names = ['lable'],header = None)

In [4]:
###########  网络
def mynetwork(mydata,mynodes,t,b):    ##数据集 ，节点集，时间间隔，滑动多少
    ij_n = len(mynodes)
    AA = np.zeros((ij_n,ij_n),dtype = 'float32')
    G = nx.Graph()
    G.add_nodes_from(mynodes['lable'])
    for i in range(len(mydata)):
        if mydata['time'][i] >= mydata['time'][0]+b:
            if mydata['time'][i] <= mydata['time'][0]+b+ t:
                G.add_edge(mydata['i'][i],mydata['j'][i])
            else:
                break
#        AA = nx.to_numpy_matrix(G) #邻接矩阵
    AA = np.array(nx.adjacency_matrix(G).todense())
    return AA


def mypayoff(A,P,S,N,RR):  ###邻接矩阵 博弈矩阵 策略矩阵 收益矩阵 整个网络第N次博弈
    for i in range(len(A)):
        for j in range(len(A)):
            if A[i,j] != 0:
                if S[N,i] == 0 :
                    if S[N,j] == 0:
                        RR[i] = RR[i] + P[0,0] 
                    else:
                        RR[i] = RR[i] + P[0,1]
                else :
                    if S[N,j] == 0:
                        RR[i] = RR[i] + P[1,0] 
                    else:
                        RR[i] = RR[i] + P[1,1]
    return RR

#### 博弈次数
def gamen(A,gn):   ###邻接矩阵，博弈次数矩阵
    for i in range(len(A)):
        if np.count_nonzero(A[i]) != 0:
            gn[i] = gn[i] + 1
    return gn
#gn =  gamen(A,np.zeros(len(A)))
#print(gn)
#np.count_nonzero(A[i])
#str(A[i]).count('1')

#####将合作概率转为策略矩阵
def stra(coor,N,S,A):
    for i in range(len(A)):
        if coor[i]<0 :
            S[N][i] = np.random.binomial(n=1, p=(1-(-coor[i])))
        else:
            S[N][i] = np.random.binomial(n=1, p=(1-coor[i]))
    return S
def stra1(coor,N,S,n):
    for i in range(n):
        S[N][i] = np.random.binomial(n=1, p=(1-coor[i]))
    return S

def upfunction(S,N,A):
    up = 0
    N_lj = np.zeros(len(A),dtype = 'float32')
    N_c = np.zeros(len(A),dtype = 'float32')
    up1 = 0
    for k in range(len(A)):
        if np.count_nonzero(A[k]) != 0:
            N_lj[N] = N_lj[N]+1  ##参与博弈的人数
            if S[N,k] == 0:
                N_c[N] = N_c[N]+1  ##博弈中选择合作的人数
    up = N_c[N]/N_lj[N]  ##### 合作者的比例      
    return up


def upfunction1(S,N,A):
    up = np.zeros(len(A),dtype = 'float32')
    N_lj = np.zeros(len(A),dtype = 'float32')
    N_c = np.zeros(len(A),dtype = 'float32')
    for k in range(len(A)):
        N_lj[k] = np.count_nonzero(A[k,])  ##与k博弈的人数
        if N_lj[k] != 0:
            for h in range(len(A)):
                if A[k,h] != 0:
                    if S[N,h] == 0:
                        N_c[k] = N_c[k]+1
                    else:
                        N_c[k] = N_c[k]
            up[k] = N_c[k]/N_lj[k]
        else:
            up[k] = 0 
    return up

def ec(A,x):
    CC = P_c = np.zeros(len(S),dtype = 'float32')   #100次合作频率
    CR = R = np.zeros(len(S),dtype = 'float32')    #100次总收益
    for i in range(len(S)):
        for j in range(len(x)):
            xcc[j] = (A[i][j]*x[j])/np.count_nonzero(A[i,])
        xc[i] = np.sum(xc[j]) 
    return xc


In [ ]:
###学习函数 m 相同

data_n = 501
ij_n = len(ij)
CC = P_c = np.zeros((30,data_n))   #100次合作频率
CR = R = np.zeros((30,ij_n))    #100次总收益
mu = 0.5
for lamdd in range(0,100,2):
    lamd = lamdd *0.1
    for mm in [2]:   ####  mm为诱惑参数
        for cc in range(30):
            cp = np.zeros((data_n+1,ij_n),dtype = 'float32')
            S = np.zeros((data_n+1,ij_n),dtype = 'float32')
            x = np.zeros(ij_n)
            for i in range(ij_n):
                x[i] = np.random.random(1)


            cp0 = x.reshape(1,ij_n)
            cp[0,] = cp0
            S = stra1(cp[0,],0,S,ij_n)

            m = 0.1
            R = np.zeros((data_n,ij_n),dtype = 'float32')                       
            RR = np.zeros((data_n,ij_n),dtype = 'float32')
            R_c = np.zeros(ij_n,dtype = 'float32')
            R_h = np.zeros(ij_n,dtype = 'float32')
            gn = np.zeros(ij_n,dtype = 'float32')
            N_lj = np.zeros(data_n,dtype = 'float32')
            N_c = np.zeros(data_n,dtype = 'float32')
            P_c = np.zeros(data_n,dtype = 'float32')
            bbb = 1 + mm /10
            P = np.array([[1,0],[bbb,0]])
            t = 900   ###一次选取30分钟的数据
            b = 60 #滑动5分钟

            for N in range(data_n):
                A = mynetwork(SFHH,ij,t,N*b)
                R[N] = mypayoff(A,P,S,N,np.zeros(ij_n))
                gn = gamen(A,gn)

                for i in range(len(A)):
                    if S[N,i] == 0:
                        R_c[i] = R[N,i] + R_c[i]
                R_h = (np.sum(R,keepdims = True,axis = 0)).reshape(ij_n,1) 

                for k in range(len(A)):
                    if np.count_nonzero(A[k]) != 0:
                        N_lj[N] = N_lj[N]+1  ##参与博弈的人数
                        if S[N,k] == 0:
                            N_c[N] = N_c[N]+1  ##博弈中选择合作的人数
                if N_lj[N] == 0:
                    P_c[N] = -1    #####没有连接
                else:
                    P_c[N] = N_c[N]/N_lj[N]  ##### 合作者的比例


                ##随机选取一个ii
                ii = random.randint(0,ij_n-1)
                while np.count_nonzero(A[ii,]) == 0:
                    ii = random.randint(0,ij_n-1)
                ##随机选取ii的邻居 jj
                G = nx.Graph(A)
                SN = G[ii]
                jj = random.choice(list(SN.keys()))

                ###模仿概率
                if S[N,jj] == 0:
                    try:
                        w = 1/(1+math.exp((R[N,ii]-R[N,jj])*10))
                    except OverflowError:
                        w = 0
                else:
                    try:
                        w = math.exp((R[N,ii]-R[N,jj])*10)/(1+math.exp((R[N,ii]-R[N,jj])*10))
                    except OverflowError:
                        w = 1


                ####更新合作概率
    #                cp = np.append(cp, cp[N], axis=0)
                cp[N+1]= cp[N]
                try:
                    nmd = 1-(2/(math.exp(gn[ii]*lamd)+math.exp(-gn[ii]*lamd)))
                except OverflowError:
                    nmd = 1
                if R_c[ii] == 0 or R_h[ii] == 0:
                    cp[N+1,ii] = (1-P_c[N])*w+P_c[N]* cp[0][ii]
                else:
                    cp[N+1,ii] = (nmd*(R_c[ii]/R_h[ii]) +(1-nmd)*w)*(1-P_c[N])+P_c[N]* cp[0][ii]

                #####更新策略
                if cp[N+1,ii] < 0:
                     cp[N+1,ii] = 0
                if cp[N+1,ii] > 1:
                     cp[N+1,ii] = 1
                S[N+1] = S[N]
                S[N+1,ii] = np.random.binomial(n=1, p=(1-(cp[N+1,ii])))

            CC[cc] = P_c
            CR[cc] = (np.sum(R,keepdims = True,axis = 0)).reshape(ij_n,)        

        filenames_out = 'E:\\Desktop\\毕业论文\\论文1\\202304结果\\学习函数\\相同'
        domain2 = os.path.abspath(filenames_out)  # 处理完文件保存地址
        fname2 = ('m = '+str(lamd)+'CC.csv','m = '+str(lamd)+'CR.csv')
        RRR = pd.DataFrame(CC)
        RRR.to_csv(os.path.join(domain2, fname2[0]))
        ggn = pd.DataFrame(CR)
        ggn.to_csv(os.path.join(domain2, fname2[1]))